In [202]:
import getpass
import pandas as pd
import re
import smtplib, ssl
import time 

from optparse import OptionParser
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication

In [212]:
# Data Wrangling

def parse_text_file(path, file_name):
    """
        Extract frequency and power levels from text files
    """
    meta_data = []
    signal_dict = {}

    with open(path+file_name) as f:
        for index, line in enumerate(f):
            
            # Check if meta-data or signal data
            if line.find(":") != -1:
                
                # Split labels and values 
                label, value = line.strip().split(":")

                # If value contains scale (e.g. Hz, remove from value and add to label)
                if value.find(",") != -1:
                    value, scale = value.split(",")
                    label = label + " (" + scale + ")"

                # Convert numbers where required 
                try:
                    value = float(value)
                except:
                    value = value.strip()

                meta_data.append([label, value])

            else:
                
                # When Frequency and Power levels are found, read rest of file instead of per line
                header = line.strip().split(",")
                levels = [x.strip().split(",") for x in f.readlines()]
                levels = {header[i]: [float(x[i]) for x in levels] for i in range(len(header))}
                break

    
    signal_dict[file_name] = {**dict(meta_data), **levels}
    return signal_dict

In [213]:
# Alerts processes 

def create_email(email_sub, email_from, email_to, email_body):
    """
        Generate message container
    """
    
    # Meta Data
    msg = MIMEMultipart('related')
    msg['Subject'] = email_sub
    msg['From'] = email_from
    msg['To'] = email_to

    # Attach data from alert
    text = MIMEText(email_body)
    msg.attach(text)
    
    return msg 


def send_email(smtp_server, port, email_from, email_password, msg):
    """
        Log into smtp server and send email
    """
    
    try:
        # Log into server
        server = smtplib.SMTP(smtp_server, port)
        server.ehlo()
        server.starttls()
        server.login(sender_email, email_password)

        # Email
        server.sendmail(msg["From"], msg["To"], msg.as_string())

    except Exception as e:
        # Print any error messages to stdout
        print(e)
    finally:
        server.quit() 
        

def alert_system(alert_data):
    """
        Create and send email once alert is actived. Attached alert data to email. 
    """
    
    # Server Credentials
    smtp_server = "smtp.gmail.com"
    port = 587  
    email_from = "ilab1.testing@gmail.com"
    email_password = getpass.getpass("Type your password and press enter: ") 

    # Email details 
    distrib_list = ["ilab1.testing@gmail.com", "ilab1.testing@gmail.com"]
    email_sub = "TEST: Alerts" 
    email_to = ", ".join(distrib_list)

    # Send
    msg = create_email(email_sub, email_from, email_to, alert_data)
    send_email(smtp_server, port, email_from, email_password, msg)

    
def constant_processing(delay=10, n=6):
    """
        Handle data updates
    """
    
    counter = 0

    while True:
        flag = False 
    
        # TODO: Process new file or datastream 
        
        if counter == 4: flag = True  # TESTING: Forcing an alert
            
        # Check and send alert 
        if flag: 
            print("Alert generating...")
            alert_time = time.strftime("%H:%M:%S %d/%m/%y", time.localtime())
            alert_system("Alert testing message. Generated at: {}".format(alert_time))
        
        # Delay until next run 
        time.sleep(delay)

        # Exit condition to avoid infinite loop
        counter += 1 
        if counter == n:
            print("Run {} times, ending process.".format(counter))
            break

In [2]:
# Parse file 
path = ""
text_file = "20210410150006.txt"

signal_dict = parse_text_file(path, text_file)

# TO-DO: read and merge multiple files

NameError: name 'parse_text_file' is not defined

In [211]:
# Check and alert
constant_processing()

Alert generating...
Type your password and press enter: ········
Run 6 times, ending process.
